In [2]:
!pip install datasets evaluate accelerate
!pip install git+https://github.com/huggingface/transformers

^C


Connect Google Drive and load the training file.

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import shutil
shutil.copy("/content/drive/MyDrive/Masterarbeit/fantastic-umbrella/run_glue_no_trainer_modded.py", "/content/run_glue_no_trainer_modded.py")

ModuleNotFoundError: No module named 'google.colab'

In [3]:
import os
import torch
print(f'torch Version {torch.__version__}')
import transformers
print(f'transformers Version {transformers.__version__}')

torch Version 2.0.0
transformers Version 4.31.0.dev0


  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers 'C:\Users\Valentin\AppData\Local\Temp\pip-req-build-m_k__rwd'


  Cloning https://github.com/huggingface/transformers to c:\users\valentin\appdata\local\temp\pip-req-build-m_k__rwd
  Resolved https://github.com/huggingface/transformers to commit 4f85aaa6c9714a6b2bbc2cdfd2399d9e4ace966f
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'


Declare all possible parameters to be examinated.

In [4]:
possible_values = {
    "task" : ['wnli', 'rte'],
    "learning_rate" : [5e-5, 1e-5],
    "hidden_dropout" : [0.1, 0.5, 0.8],
    "use_pretrain_dropout_in_first_pass" : [True, False],
    "train_data_frac" : [1, 0.5, 0.1],
    "use_modded" : [True, False],
    "num_train_epochs" : [5, 10, 20]
}

Create an exhaustive list of all parameter combinations.

In [5]:
from itertools import product
keys, values = zip(*possible_values.items())
permutations_dicts = [dict(zip(keys, v)) for v in product(*values)]
print(len(permutations_dicts))
permutations_dicts

432


[{'task': 'wnli',
  'learning_rate': 5e-05,
  'hidden_dropout': 0.1,
  'use_pretrain_dropout_in_first_pass': True,
  'train_data_frac': 1,
  'use_modded': True,
  'num_train_epochs': 5},
 {'task': 'wnli',
  'learning_rate': 5e-05,
  'hidden_dropout': 0.1,
  'use_pretrain_dropout_in_first_pass': True,
  'train_data_frac': 1,
  'use_modded': True,
  'num_train_epochs': 10},
 {'task': 'wnli',
  'learning_rate': 5e-05,
  'hidden_dropout': 0.1,
  'use_pretrain_dropout_in_first_pass': True,
  'train_data_frac': 1,
  'use_modded': True,
  'num_train_epochs': 20},
 {'task': 'wnli',
  'learning_rate': 5e-05,
  'hidden_dropout': 0.1,
  'use_pretrain_dropout_in_first_pass': True,
  'train_data_frac': 1,
  'use_modded': False,
  'num_train_epochs': 5},
 {'task': 'wnli',
  'learning_rate': 5e-05,
  'hidden_dropout': 0.1,
  'use_pretrain_dropout_in_first_pass': True,
  'train_data_frac': 1,
  'use_modded': False,
  'num_train_epochs': 10},
 {'task': 'wnli',
  'learning_rate': 5e-05,
  'hidden_dropou

Load the Tensorboard for live tracking.

In [2]:
# Load the TensorBoard notebook extension
%load_ext tensorboard
%tensorboard --logdir "/content/drive/MyDrive/Masterarbeit/fantastic-umbrella/runs/"

Launching TensorBoard...

In [ ]:
%load_ext tensorboard
%tensorboard --logdir "G:\\Meine Ablage\\Masterarbeit\\fantastic-umbrella\\runs"

Construct and execute the script command for every parameter combination.

In [6]:
import datetime
import os
for iteration, parameters in enumerate(permutations_dicts):
    print(f'##################################### Current iteration: {iteration} #####################################')
    base_command = 'python run_glue_no_trainer_modded.py '
    argument_list = ["--model_name_or_path", "bert-base-cased",
                     "--max_length", "128",
                     "--per_device_train_batch_size", "32",
                     "--seed", "0",
                     "--with_tracking",
                     "--report_to", "tensorboard"
                    ]



    argument_list = argument_list + ["--task_name", f'{parameters["task"]}']
    argument_list = argument_list + ["--num_train_epochs", f'{parameters["num_train_epochs"]}']
    argument_list = argument_list + ["--learning_rate", f'{parameters["learning_rate"]}']
    argument_list = argument_list + ["--train_data_frac", f'{parameters["train_data_frac"]}']
    argument_list = argument_list + ["--hidden_dropout", f'{parameters["hidden_dropout"]}']


    if parameters["use_modded"]:
        argument_list.append("--use_modded")

    if parameters["use_pretrain_dropout_in_first_pass"]:
        argument_list.append("--use_pretrain_dropout_in_first_pass")
    
    folder_name = 'run-{date:%Y%m%d_%H%M%S}'.format( date=datetime.datetime.now() )
    output_dir = f"/content/drive/MyDrive/Masterarbeit/fantastic-umbrella/runs/{folder_name}"
    argument_list = argument_list + ["--output_dir",output_dir]
    argument_string = ' '.join(argument_list)
    instruction = base_command + argument_string

    # run the script with the desired arguments
    # os.system(instruction) # vscode style
    !{instruction} # colab style

##################################### Current iteration: 0 #####################################
